In [4]:
from lib.data_handling import load_data, CustomSpectralPipeline
from lib.reproduction import masks, oxide_ranges, spectrometer_wavelength_ranges, major_oxides
from lib.norms import Norm3Scaler

In [2]:
data = load_data(
    "../data/data/calib/calib_2015/1600mm/pls/",
    1
)

Loading data: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]


In [5]:
pipeline = CustomSpectralPipeline(
    masks=masks,
    major_oxides=major_oxides,
    composition_data_loc="../data/data/calib/ccam_calibration_compositions.csv",
)

processed_data = pipeline.fit_transform(data)
processed_data.head()

Transforming samples: 100%|██████████| 1/1 [00:00<00:00, 11.47it/s]


,246.688,246.741,246.79401,246.847,246.89999,246.953,247.007,247.06,247.11301,247.166,...,848.89642,SiO2,TiO2,Al2O3,FeOT,MgO,CaO,Na2O,K2O,Sample Name
0,3.998589e+11,3.623102e+11,3.125834e+11,2.548143e+11,2.384109e+11,2.535727e+11,2.867055e+11,3.181519e+11,3.228308e+11,3.108220e+11,...,2.293541e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
1,3.101115e+11,2.760840e+11,2.294153e+11,1.789714e+11,1.602546e+11,1.737365e+11,2.033433e+11,2.307903e+11,2.380900e+11,2.286484e+11,...,2.224658e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
2,4.784395e+11,4.364859e+11,3.787261e+11,3.007351e+11,2.869929e+11,3.060502e+11,3.437366e+11,3.862550e+11,3.896103e+11,3.750934e+11,...,2.239592e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
3,5.402266e+11,4.974165e+11,4.266432e+11,3.354670e+11,3.232402e+11,3.433198e+11,3.847175e+11,4.298031e+11,4.313859e+11,4.173089e+11,...,1.634240e+09,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
4,4.925927e+11,4.457917e+11,3.862441e+11,3.063318e+11,2.987273e+11,3.191648e+11,3.567172e+11,3.909940e+11,3.978301e+11,3.836622e+11,...,2.234164e+10,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399


In [6]:
scaler = Norm3Scaler(wavelength_ranges=spectrometer_wavelength_ranges)
scaled = scaler.fit_transform(processed_data)
scaled[:5]

,246.688,246.741,246.79401,246.847,246.89999,246.953,247.007,247.06,247.11301,247.166,...,848.89642,SiO2,TiO2,Al2O3,FeOT,MgO,CaO,Na2O,K2O,Sample Name
0,0.000191,0.000173,0.000149,0.000122,0.000114,0.000121,0.000137,0.000152,0.000154,0.000148,...,0.000180,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
1,0.000148,0.000132,0.000109,0.000085,0.000076,0.000083,0.000097,0.000110,0.000114,0.000109,...,0.000175,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
2,0.000228,0.000208,0.000181,0.000144,0.000137,0.000146,0.000164,0.000184,0.000186,0.000179,...,0.000176,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
3,0.000258,0.000237,0.000204,0.000160,0.000154,0.000164,0.000184,0.000205,0.000206,0.000199,...,0.000013,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399
4,0.000235,0.000213,0.000184,0.000146,0.000143,0.000152,0.000170,0.000187,0.000190,0.000183,...,0.000176,51.03,1,13.83,9.93,6.33,10.7,2.01,0.6,jsc1399


In [12]:
# test if the scaler is working
import numpy as np

drop_cols = major_oxides + ['Sample Name']

# assert sum of each of the three ranges is 3 using spectrometer_wavelength_ranges
def test_norm3_scaler(test_df):
    # Simulated Data
    df = test_df

    # Spectrometer wavelength ranges
    spectrometer_wavelength_ranges = {
        "UV": (223.4, 325.97),
        "VIO": (381.86, 471.03),
        "VNIR": (494.93, 927.06),
    }

    # Initialize and fit-transform the scaler
    scaler = Norm3Scaler(spectrometer_wavelength_ranges)
    scaler.fit(df)
    transformed_df = scaler.transform(df.copy())

    numerical_df = transformed_df.drop(drop_cols, axis=1)

    # Verify the sum of each range is approximately 1
    for key, (start, end) in spectrometer_wavelength_ranges.items():
        selected_columns = [col for col in numerical_df.columns if start <= float(col) <= end]
        range_sum = numerical_df[selected_columns].sum().sum()
        assert np.isclose(range_sum, 1.0), f"Sum for range {key} is not close to 1.0 but is {range_sum}"
        print(f"Sum for range {key} is close to 1.0 ({range_sum})")

# Run the test
test_norm3_scaler(scaled)


Sum for range UV is close to 1.0 (1.0)
Sum for range VIS is close to 1.0 (1.0)
Sum for range VNIR is close to 1.0 (1.0000000000000002)
